## Introduction
I Name:Siphesihle Mbonambi Student No: 220108601, confirm that by submitting this document, that the solutions in this notebook are a result of my own work. 

**AI generated code will be subject to a mark of 0.**

Submit a Python notebook with your name surname_student number_Assignment 1

DUE 9 June 2024, 23:59
 
WEIGHTING IS 50% of Course Mark 

## Movielens Project

The goal of this MovieLens Project is to create a movie recommendation model able to predict movie ratings with a root mean squared error (RMSE) of less than 0.86490 compared to actual movie ratings. Data is sourced from the MovieLens dataset and divided into two datasets, each containing the variables userId, movieId, rating, timestamp, title, and genres:

•	train, with approximately 9 million observations.

•	validation, with about 1 million observations.



## Download the dataset and split

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import requests
from io import BytesIO
from zipfile import ZipFile

# Download movielens 

response = requests.get("https://files.grouplens.org/datasets/movielens/ml-10m.zip")
zip_file = ZipFile(BytesIO(response.content))
zip_file.extractall('data/')  

# Read and process ratings 
ratings = pd.read_csv('data/ml-10M100K/ratings.dat', delimiter='::', engine='python',
                      names=['userId', 'movieId', 'rating', 'timestamp'], header=None)

# Read and process movies 
movies = pd.read_csv('data/ml-10M100K/movies.dat', delimiter='::', engine='python',
                     names=['movieId', 'title', 'genres'], header=None)

# Merge ratings and movies 
movielens = pd.merge(ratings, movies, on='movieId')

# Create a validation set, which is 10% MovieLens dataset
train, validation = train_test_split(movielens, test_size=0.1, random_state=1)

# Ensure userId and movieId in the validation set are also in the train set
validation = validation[validation['movieId'].isin(train['movieId']) & validation['userId'].isin(train['userId'])]

# Add back any removed rows into the training set
removed = validation[~(validation['movieId'].isin(train['movieId']) & validation['userId'].isin(train['userId']))]
train = pd.concat([train, removed])

# Clean up 
try:
    zip_file.close()
except:
    pass

print("Setup complete.")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import requests
from io import BytesIO
from zipfile import ZipFile

# Download movielens

response = requests.get("https://files.grouplens.org/datasets/movielens/ml-10m.zip")
zip_file = ZipFile(BytesIO(response.content))
zip_file.extractall('data/')

# Read and process ratings
ratings = pd.read_csv('data/ml-10M100K/ratings.dat', delimiter='::', engine='python',
                      names=['userId', 'movieId', 'rating', 'timestamp'], header=None)

# Read and process movies
movies = pd.read_csv('data/ml-10M100K/movies.dat', delimiter='::', engine='python',
                     names=['movieId', 'title', 'genres'], header=None)

# Merge ratings and movies
movielens = pd.merge(ratings, movies, on='movieId')

# Create a validation set, which is 10% MovieLens dataset
train, validation = train_test_split(movielens, test_size=0.1, random_state=1)

# Ensure userId and movieId in the validation set are also in the train set
validation = validation[validation['movieId'].isin(train['movieId']) & validation['userId'].isin(train['userId'])]

# Add back any removed rows into the training set
removed = validation[~(validation['movieId'].isin(train['movieId']) & validation['userId'].isin(train['userId']))]
train = pd.concat([train, removed])

# Clean up
try:
    zip_file.close()
except:
    pass

print("Setup complete.")

In [ ]:
## Section 1: Data Exploration (15 marks)

1.	How many rows and columns are there in the movielens dataset? (3 marks)

In [ ]:
movielens.shape

2. How many zeros were given as ratings in the movielens dataset? (2 marks)

In [ ]:
movielens[movielens['rating']==0].shape[0]

3.	How many threes were given as ratings in the movielens dataset? (2 marks)

In [ ]:
movielens[movielens['rating']==3.0].shape[0]

4. How many different movies are in the movielens dataset? (2 marks)

In [ ]:
movielens['movieId'].nunique()

5.	How many different users are in the movielens dataset? (2 marks)

In [ ]:
movielens['userId'].nunique()

6.	Which movie has the greatest number of ratings? (5 marks)

In [ ]:
x = movielens['movieId'].value_counts()
y=x.idxmax()
z=x.max()
print(f'Movie iD {y}')
print(f'Number of ratings {z}')

## Section 2

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn import preprocessing 

In [ ]:
# Checked the first 5 rows to haev an idea of what the data looks like
movielens.head()

In [ ]:
# CHekced the info of the dataset to also find out the dtype of each column
movielens.info()

In [ ]:
# Wanted to find out the basics stats such as your mean, std, min and max values
movielens.describe()

In [ ]:
# Checked for any null values
movielens.isnull().sum()

In [ ]:
# Checked for any duplicates
movielens.duplicated().sum()

In [ ]:
# Changed the timestamp column to dateTime
movielens['timestamp'] = pd.to_datetime(movielens['timestamp'], unit = 's')

In [ ]:
movielens['year']=movielens['timestamp'].dt.year
movielens['month']=movielens['timestamp'].dt.month

In [ ]:
movielens

In [ ]:
#Visualized the distribution of the movie ratings
x=movielens['rating'].value_counts().sort_index()
plt.bar(x.index, x.values, color='blue')
plt.xlabel('Rating')
plt.ylabel('Count of Ratings')
plt.title('Movie Rating Distribtuion')
plt.show()

In [ ]:
#Checked which movie has the most rating
movielens['title'].mode()

In [ ]:
# Checked which user ID has done the most ratings
movielens['userId'].mode()

In [ ]:
# Checked which movie has the most 5 star ratings
q=movielens[movielens['rating'] ==5]
fivestar= q['title'].value_counts()
moststar=fivestar.idxmax()
moststarcount=fivestar.max()

print(moststar)
print(moststarcount)

In [ ]:
# Checked with movie has the worst ratings
s=movielens[movielens['rating'] ==0.5]
onestar= s['title'].value_counts()
leaststar=onestar.idxmax()
leaststarcount=fivestar.max()

print(leaststar)
print(leaststarcount)

In [ ]:
# Checked the average rating of the movie with the most ratings
x = movielens[movielens['movieId']== 296]
x['rating'].mean()

In [ ]:
# Checked how many movies had a rating of 0.5
g= movielens[movielens['rating']<1]
g.info()

In [ ]:
# Checked for any outliers in rating
plt.figure(figsize=(5,5))
sns.boxplot(y=movielens['rating'])
plt.show()

# Linear Regression Model


In [ ]:
movielens_copy = movielens.copy()

In [ ]:
# Calculating the Correlation/ Made each of the genres a variable
genre_dumie = movielens_copy['genres'].str.get_dummies('|')
movielens_copy = pd.concat([movielens, genre_dumie], axis=1)

In [ ]:
# Used the label encoder from scikit-learn to turn the categorical values in title and genre to numerical labels
label_encoder = preprocessing.LabelEncoder()

movielens_copy['title'] = label_encoder.fit_transform(movielens_copy['title'])

movielens_copy['genres'] = label_encoder.fit_transform(movielens_copy['genres'])

In [ ]:
# I then plotted the heat map
plt.figure(figsize=(17,8))
dataplot = sns.heatmap(movielens_copy.corr(), cmap="YlGnBu", annot=True) 

# Building the model

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer


In [ ]:
# Calculated the 25th percentile and 75th percentile for rating as there are outliers
q1 = movielens['rating'].quantile(0.25)
q3 = movielens['rating'].quantile(0.75)

# I then Calculated the IQR
iqr = q3 - q1

# I then determined the outlier bounds
lower = q1 - 1.5 * iqr
upper = q3 + 1.5 * iqr

# Identify outliers
outliers = movielens[(movielens['rating'] < lower) | (movielens['rating'] > upper)]

movielens = movielens[(movielens['rating'] >= lower) & (movielens['rating'] <= upper)]

In [ ]:
# Went on to define the initial features and target variable
original_columns = ['movieId', 'userId']
target_variable = 'rating'

In [ ]:
# Added additional features for movieId such as mean, median, mode
movie_stats = movielens.groupby('movieId')['rating'].agg(['mean', 'median', 'std', 'count']).reset_index()
movie_stats.columns = ['movieId', 'movie_mean_rating', 'movie_median_rating', 'movie_std_rating', 'movie_rating_count']


In [ ]:
# Added additional features for userId too
user_stats = movielens.groupby('userId')['rating'].agg(['mean', 'median', 'std', 'count']).reset_index()
user_stats.columns = ['userId', 'user_mean_rating', 'user_median_rating', 'user_std_rating', 'user_rating_count']


In [ ]:
# I then merged these features for userId and movieId to the original dataset
movielens = pd.merge(movielens, movie_stats, on='movieId')
movielens = pd.merge(movielens, user_stats, on='userId')


In [ ]:
# Combined all the features that I would be using 
new_columns = original_columns + ['movie_mean_rating', 'movie_median_rating', 'movie_std_rating', 'movie_rating_count',
                                  'user_mean_rating', 'user_median_rating', 'user_std_rating', 'user_rating_count']


In [ ]:
# Chose the columns that would be required for the model
X = movielens[new_columns]
y = movielens[target_variable]

In [ ]:
# Used the mean to impute any missing values
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)


In [ ]:
# I then normalized the features
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X_imputed)

In [ ]:
# Split the dataset into training and test sets with the test size being 10% 
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.1, random_state=0)

# Initialized the linear regression model
linear_regression = LinearRegression()


In [ ]:
# I then Fit the model on the training data
linear_regression.fit(X_train, y_train)

# Predicted on the test set
y_pred = linear_regression.predict(X_test)


In [ ]:
# Calculate metrics
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rmse


In [ ]:
results= pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
results

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()],[y_test.min(),y_test.max()],'--')
plt.xlabel('Original Ratings')
plt.ylabel('Predicted Ratings')
plt.title('Original vs Predicted Ratings')
plt.show()